In [10]:
import pandas as pd

# Brute data from different sources:
# https://ai-project-inflation.s3.amazonaws.com/
# cpi_inflation = pd.read_csv("./1810025601-CPI_and_Inflation_ydata.csv", thousands=',')
brent_crude_prices = pd.read_excel("./Brent_crude_prices.xls", sheet_name=1, header=2)
# fx_rates = pd.read_excel("./FX_RATES_MONTHLY.xlsx")
# prime_rate = pd.read_excel("./Prime_Rate_History-WOWA.xlsx")
# manufacturing_rate = pd.read_excel("./Stlouisfeb_hourmanurate.csv")
# bond_rate = pd.read_csv("./StlouisfedCAD10yearbondrate.csv")
# money_supply = pd.read_csv("./Stlousisfed_Canadain_Money_supply.csv")
# fed_fx_rates = pd.read_csv("./stlousisfedFXRates.csv")
# tsx_data = pd.read_csv("./TSX_data_for_20_years.csv")

brent_crude_prices.shape

brent_crude_prices.head()




,Date,Europe Brent Spot Price FOB (Dollars per Barrel)
0,1987-05-15,18.58
1,1987-06-15,18.86
2,1987-07-15,19.86
3,1987-08-15,18.98
4,1987-09-15,18.31


In [3]:
fire_data.shape

(17536, 43)

In [30]:
import datetime
tfp_alarm_time = datetime.datetime.strptime("2018-02-24T21:04:29", "%Y-%m-%dT%H:%M:%S")
print(tfp_alarm_time)

2018-02-24 21:04:29


In [27]:
from dateutil.parser import parse
import datetime
import pandas as pd
data = ['item_1', 'item_2', 'item_3']
df = pd.DataFrame (data, columns = ['column_name'])


result = df.apply(func)
tfp_alarm_time = datetime.datetime.strptime("Fri Sep 25 18:09:49 -1000 2009", "%Y-%m-%dT%H:%M:%S")
tfp_arrival_time = datetime.datetime.strptime(fire_data[["TFS_Arrival_Time"]], "%Y-%m-%dT%H:%M:%S")

time_frame = tfp_arrival_time - tfp_alarm_time
print(time_frame)
# datetime.datetime()
#
# a = parse("Fri Sep 25 18:09:49 -1000 2009")
# print(a)
# delayTruck = datetime.datetime(fire_data[["TFS_Alarm_Time"]]) - datetime.datetime()
  

TypeError: strptime() argument 1 must be str, not DataFrame